In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

from fastai.imports import *
np.set_printoptions(linewidth=110)

In [ ]:
import torch
import gc

def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [1]:
import pandas as pd

results_df = pd.DataFrame(columns=[''])

def track_results(experiment_name, result, reset=False):
    global results_df
    if reset:
        results_df = pd.DataFrame(columns=[''])
    if hasattr(result, '__dict__') and '__str__' in result.__dict__:
        results_df.loc[experiment_name] = result
    else:
        results_df.loc[experiment_name] = repr(result)
    return results_df

In [ ]:
cbs = GradientAccumulation(64)
learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()

In [ ]:
def show_last_text_batch(dls):
    last_batch = None
    for batch in dls.train:
        last_batch = batch
    x, y = last_batch

    # Get the vocabulary from the DataLoader
    vocab = dls.vocab

    # Function to convert indices to text
    def indices_to_text(indices):
        return ' '.join([vocab[it] for it in indices])

    # Convert and print the first input sequence in the batch
    print(indices_to_text(x[0]), ' | ', indices_to_text(x[1]))

In [1]:
import numpy as np
import pandas as pd

def display_learn_model_stats(learn):
    # Initialize the data dictionary
    data = {
        'Layer': [],
        'Weights': [],
        'Conv Size': [],
        'Activation Cells': [],
        'Multiplications': []
    }

    # Get initial conv_size from the data loader
    conv_size = dls.one_batch()[0].shape[-1]

    # Store the previous activation cells, initialized to the input size
    prev_activation_cells = conv_size ** 2

    # Iterate through the model layers
    for i, layer in enumerate(learn.model.children()):
        # Check if the layer is a Sequential or Conv2d
        if isinstance(layer, nn.Sequential):
            conv_layer = layer[0]
        elif isinstance(layer, nn.Conv2d):
            conv_layer = layer
        else:
            continue
        
        if not isinstance(conv_layer, nn.Conv2d):
            continue
        
        layer_name = f'Layer {i+1}'
        data['Layer'].append(layer_name)
        
        # Calculate weights (number of parameters in the convolutional layer)
        weights = np.prod(conv_layer.weight.shape) + conv_layer.bias.shape[0]
        data['Weights'].append(weights)
        
        # Update conv_size based on the current layer parameters
        kernel_size = conv_layer.kernel_size[0]
        stride = conv_layer.stride[0]
        padding = conv_layer.padding[0]
        
        conv_size = (conv_size - kernel_size + 2 * padding) // stride + 1
        data['Conv Size'].append(conv_size)
        
        # Calculate activation cells (number of cells in the feature map)
        activation_cells = conv_size ** 2
        data['Activation Cells'].append(activation_cells)
        
        # Calculate multiplications
        multiplications = weights * prev_activation_cells
        data['Multiplications'].append(multiplications)
        
        # Update prev_activation_cells for the next iteration
        prev_activation_cells = activation_cells

    # Create a DataFrame from the data dictionary
    return pd.DataFrame(data)


display_learn_model_stats(learn)

NameError: name 'learn' is not defined

In [1]:
import torch

def torch_show_img(tns, title=''):
    if tns.ndim == 1:  # If rank 1, reshape to 2D assuming it's a 28x28 image
        tns = tns.reshape(28, 28)
    show_image(torch.tensor(tns), title=title)

In [4]:
import matplotlib.pyplot as plt

def np_show_img(array, title=''):
    if array.ndim == 1:  # If rank 1, reshape to 2D
        array = array.reshape(28, 28)
    plt.imshow(array, cmap='gray')
    plt.axis('off')
    plt.show()

In [2]:
# reshape image tensor
import torch

image_tns = torch.randn((2000, 3, 28, 28))
m, c, h, w = image_tns.shape
image_tensor = image_tns.view(m, c, h * w)
image_tensor.shape

torch.Size([2000, 3, 784])